In [ ]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/google/lightweight_mmm.git

  Cloning https://github.com/google/lightweight_mmm.git to /tmp/pip-req-build-b447w83d
  Running command git clone --filter=blob:none --quiet https://github.com/google/lightweight_mmm.git /tmp/pip-req-build-b447w83d
  Resolved https://github.com/google/lightweight_mmm.git to commit 4406aaa77bddc5b0d73d31c6cf4f2ace03f3ffda
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
from lightweight_mmm import preprocessing, lightweight_mmm, plot, optimize_media
import jax.numpy as jnp
from sklearn.metrics import mean_absolute_percentage_error
import jax.numpy as jnp
import numpyro
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ins_data = pd.read_csv("/content/instagram_processed_data.csv", usecols=["Impressions", "Reach", "Engaged Users", "Fan Growth", "Views", "Cost", "Consideration"])

In [ ]:
ins_data.describe

<bound method NDFrame.describe of      Impressions  Reach  Engaged Users  Fan Growth  Views    Cost  Awareness
0           7705   7512            334           0      0  23.115   0.001542
1          15187  14754            356          12   3168  45.561   0.003029
2          12381  11771            451           0      0  37.143   0.002417
3          18267  17524            614          20   7571  54.801   0.003598
4          12898  12659            337           0      0  38.694   0.002599
..           ...    ...            ...         ...    ...     ...        ...
965        10599   9874           1128           2      0  31.797   0.002027
966         8171   7596            944           1      0  24.513   0.001559
967         4030   3764            330           0   1338  12.090   0.000773
968         4761   3527            321           1      0  14.283   0.000724
969        10775   9865            726           6   3371  32.325   0.002025

[970 rows x 7 columns]>

In [ ]:
ins_data.shape

(970, 7)

In [ ]:
ins_data = ins_data[:485]

In [ ]:
media_data = ins_data[["Impressions"]].to_numpy()

cost_data = ins_data[["Cost"]].to_numpy()

sales_data = ins_data[["Consideration"]].to_numpy()

extra_features_data = ins_data[["Reach", "Engaged Users", "Fan Growth", "Views"]].to_numpy()

In [ ]:
media_data

array([[  7705],
       [ 15187],
       [ 12381],
       [ 18267],
       [ 12898],
       [ 13656],
       [ 32909],
       [  7544],
       [ 31033],
       [ 46111],
       [ 31947],
       [ 15808],
       [ 43851],
       [  6377],
       [  9316],
       [  5653],
       [  7521],
       [ 10099],
       [  8184],
       [ 11394],
       [  5777],
       [ 18575],
       [  8523],
       [ 11585],
       [  7879],
       [  3741],
       [  3824],
       [  7360],
       [  3051],
       [  4456],
       [  7865],
       [  5597],
       [  6132],
       [  4365],
       [  4741],
       [  6471],
       [  5740],
       [ 14601],
       [ 78929],
       [ 17869],
       [  6568],
       [  6735],
       [  7970],
       [  3725],
       [252592],
       [  6039],
       [ 11069],
       [  5866],
       [ 14278],
       [ 10159],
       [  4734],
       [  6222],
       [ 11604],
       [  9256],
       [  4465],
       [  4929],
       [  4219],
       [156508],
       [  5975

In [ ]:
extra_features_data

array([[ 7512,   334,     0,     0],
       [14754,   356,    12,  3168],
       [11771,   451,     0,     0],
       ...,
       [ 3764,   330,     0,  1338],
       [ 3527,   321,     1,     0],
       [ 9865,   726,     6,  3371]])

In [ ]:
media_data

array([[  7705],
       [ 15187],
       [ 12381],
       [ 18267],
       [ 12898],
       [ 13656],
       [ 32909],
       [  7544],
       [ 31033],
       [ 46111],
       [ 31947],
       [ 15808],
       [ 43851],
       [  6377],
       [  9316],
       [  5653],
       [  7521],
       [ 10099],
       [  8184],
       [ 11394],
       [  5777],
       [ 18575],
       [  8523],
       [ 11585],
       [  7879],
       [  3741],
       [  3824],
       [  7360],
       [  3051],
       [  4456],
       [  7865],
       [  5597],
       [  6132],
       [  4365],
       [  4741],
       [  6471],
       [  5740],
       [ 14601],
       [ 78929],
       [ 17869],
       [  6568],
       [  6735],
       [  7970],
       [  3725],
       [252592],
       [  6039],
       [ 11069],
       [  5866],
       [ 14278],
       [ 10159],
       [  4734],
       [  6222],
       [ 11604],
       [  9256],
       [  4465],
       [  4929],
       [  4219],
       [156508],
       [  5975

In [ ]:
media_data_train = media_data[:388]
media_data_test = media_data[388:]
target_data_train = sales_data[:388]
target_data_test = sales_data[388:]
cost_data_train = cost_data[:388].sum(axis=0)
#cost_data_train = cost_data_train.reshape(-1, 1)
cost_data_test = cost_data[388:].sum(axis=0)
#cost_data_test = cost_data_test.reshape(-1, 1)
extra_features_data_train = extra_features_data[:388]
extra_features_data_test = extra_features_data[388:]

In [ ]:
cost_data_train

array([6983.412])

In [ ]:
media_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
target_scaler = preprocessing.CustomScaler(
    divide_operation=jnp.mean)
cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
extra_features_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)

In [ ]:
media_data_train_scaled = media_scaler.fit_transform(media_data_train)
target_train_scaled = target_scaler.fit_transform(target_data_train)
costs_scaled = cost_scaler.fit_transform(cost_data_train)
extra_features_scaled = extra_features_scaler.fit_transform(extra_features_data_train)

media_data_test_scaled = media_scaler.transform(media_data_test)
extra_features_test_scaled = extra_features_scaler.fit_transform(extra_features_data_test)

In [ ]:
costs_scaled

Array([1.], dtype=float32)

In [ ]:
media_data_train_scaled

Array([[1.2842748e+00],
       [2.5313799e+00],
       [2.0636737e+00],
       [3.0447564e+00],
       [2.1498477e+00],
       [2.2761917e+00],
       [5.4852953e+00],
       [1.2574393e+00],
       [5.1726022e+00],
       [7.6858139e+00],
       [5.3249483e+00],
       [2.6348886e+00],
       [7.3091154e+00],
       [1.0629228e+00],
       [1.5527974e+00],
       [9.4224602e-01],
       [1.2536056e+00],
       [1.6833085e+00],
       [1.3641149e+00],
       [1.8991599e+00],
       [9.6291447e-01],
       [3.0960941e+00],
       [1.4206196e+00],
       [1.9309959e+00],
       [1.3132772e+00],
       [6.2355250e-01],
       [6.3738704e-01],
       [1.2267700e+00],
       [5.0854284e-01],
       [7.4272925e-01],
       [1.3109437e+00],
       [9.3291193e-01],
       [1.0220861e+00],
       [7.2756130e-01],
       [7.9023325e-01],
       [1.0785909e+00],
       [9.5674723e-01],
       [2.4337051e+00],
       [1.3155941e+01],
       [2.9784176e+00],
       [1.0947589e+00],
       [1.122594

In [ ]:
number_warmup=1000
number_samples=1000

In [ ]:
mmm = lightweight_mmm.LightweightMMM(model_name="carryover")

In [ ]:
mmm.fit(
    media=media_data_train_scaled,
    media_prior=costs_scaled,
    target=target_train_scaled,
    extra_features=extra_features_scaled,
    number_warmup=number_warmup,
    number_samples=number_samples,
    degrees_seasonality=3,
    weekday_seasonality=True,
    seasonality_frequency=365,
    seed=1)

NameError: name 'mmm' is not defined

In [ ]:
adstock_models = ["adstock", "hill_adstock", "carryover"]
degrees_season = [1,2,3]

#adstock_models = ["hill_adstock"]
#degrees_season = [1]


for model_name in adstock_models:
    for degrees in degrees_season:
        mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
        mmm.fit(media=media_data_train_scaled,
                media_prior=costs_scaled,
                target=target_train_scaled,
                extra_features=extra_features_scaled,
                number_warmup=1000,
                number_samples=1000,
                number_chains=1,

                seed=1)

        prediction = mmm.predict(
        media=media_data_test_scaled,
        extra_features=extra_features_test_scaled,
        target_scaler=target_scaler,
        seed=1)
        p = prediction.mean(axis=0)

        mape = mean_absolute_percentage_error(target_data_test, p)
        print(f"model_name={model_name} degrees={degrees} MAPE={mape} samples={p[:3]}")

sample: 100%|██████████| 2000/2000 [08:43<00:00,  3.82it/s, 60 steps of size 4.03e-02. acc. prob=0.69]


model_name=adstock degrees=1 MAPE=43991396937483.305 samples=[0.15811203 0.15815188 0.15819016]


warmup:  39%|███▉      | 778/2000 [06:18<09:53,  2.06it/s, 831 steps of size 3.18e-02. acc. prob=0.84]


KeyboardInterrupt: 